In [1]:
from functions import *

# greeks("NVDA", '2026-01-16', 175, "put")

In [5]:
continuous_monitor("AMC")

Starting price: 1.934999942779541
Stopped monitoring


In [ ]:
greeks("AMC", '2026-01-16', 2, "call")

{'lastPrice': 0.17, 'bid': 0.15, 'ask': 0.17, 'volume': 1080}

In [3]:
option_price("AMC", '2026-01-16', 2, "call")

{'lastPrice': 0.17, 'bid': 0.15, 'ask': 0.17, 'volume': 1080}

In [2]:
# Get historical stock price
price = get_stock_price_historical('AAPL', '2024-01-15')
print(f"AAPL price on 2024-01-15: ${price}")

# Calculate theoretical option price on a historical date
option_price = option_price_historical(
    'AAPL', '2024-03-15', 150.0, 'call', '2024-01-15'
)
print(f"Theoretical call price: ${option_price}")

# Calculate Greeks on a historical date
greeks_data = greeks_historical(
    'AAPL', '2024-03-15', 150.0, 'call', '2024-01-15'
)
print(f"Historical Greeks: {greeks_data}")

# With custom implied volatility
option_price = option_price_historical(
    'AAPL', '2024-03-15', 150.0, 'call', '2024-01-15', iv=0.25
)
print(f"Price with 25% IV: ${option_price}")


AAPL price on 2024-01-15: $184.18063354492188
Theoretical call price: $35.462936939928454
Historical Greeks: {'delta': 0.9984572699653671, 'gamma': 0.00037150914325497793, 'vega': 0.0037384517644744163, 'theta': -0.02170075918108708, 'rho': 0.24400036538601527}
Price with 25% IV: $35.562442645391904


In [5]:
# Get intraday stock price
price = get_stock_price_intraday('AAPL', '2025-12-01', '14:30')
print(f"AAPL at 2:30 PM: ${price}")

# Calculate option price at a specific time
option_price = option_price_intraday(
    'AAPL', '2026-01-16', 150.0, 'call', 
    '2025-12-01', '14:30'
)
print(f"Call option price at 2:30 PM: ${option_price}")

# Calculate Greeks at a specific time
greeks = greeks_intraday(
    'AAPL', '2026-01-16', 150.0, 'call',
    '2025-12-01', '14:30'
)
print(f"Greeks at 2:30 PM: \n{greeks}")

# With custom IV
greeks = greeks_intraday(
    'AAPL', '2026-01-16', 150.0, 'call',
    '2025-12-01', '14:30', iv=0.28
)
print(f"Greeks at 2:30 PM with IV = 28%: \n{greeks}")

AAPL at 2:30 PM: $281.9100036621094
Call option price at 2:30 PM: $132.60697821767744
Greeks at 2:30 PM: 
{'delta': 1.0, 'gamma': 1.3478030356826388e-33, 'vega': 2.0184015064306322e-32, 'theta': -0.015106194099780191, 'rho': 0.18828940751633524}
Greeks at 2:30 PM with IV = 28%: 
{'delta': 0.999999999941034, 'gamma': 1.3863484026026166e-11, 'vega': 3.8905291417584803e-10, 'theta': -0.01510619421640083, 'rho': 0.18828940749505735}


In [ ]:
# Simulate buying a call option with 20% stop-loss and 50% take-profit
result = options_purchase(
    'AAPL', 
    '2026-01-16',  # Expiration
    150.0,          # Strike
    '2025-12-10',   # Purchase date
    '10:30',        # Purchase time (10:30 AM ET)
    'call',
    stoploss=20,    # -20% stop-loss
    takeprofit=50   # +50% take-profit
)

print(f"Entry: ${result['entry_price']}")
print(f"Exit: ${result['exit_price']} at {result['exit_time']}")
print(f"Reason: {result['exit_reason']}")
print(f"P/L: {result['pnl_percent']}% (${result['pnl_dollar']} per contract)")


Entry: $128.81
Stop-loss: $103.04 (-20%)
Take-profit: $193.21 (+50%)

Monitoring position from 2025-12-10 until expiration (2026-01-16)...
------------------------------------------------------------

2025-12-10:
  11:00: $128.81
  12:00: $129.07
  13:00: $128.67
  14:00: $128.77
  15:00: $129.53
  16:00: $129.26

2025-12-11:
  10:00: $126.65
  11:00: $126.35
  12:00: $128.43
  13:00: $128.05
  14:00: $128.29
  15:00: $128.62
  16:00: $128.59

2025-12-12:
  10:00: $128.38
  11:00: $128.25
  12:00: $128.36
  13:00: $129.56
  14:00: $128.46
  15:00: $128.56
  16:00: $128.88

2025-12-15:
  10:00: $124.61
  11:00: $125.98
  12:00: $125.39
  13:00: $124.51
  14:00: $123.84
  15:00: $124.27
  16:00: $124.58

2025-12-16:
  10:00: $123.62
  11:00: $123.26
  12:00: $123.65
  13:00: $122.58
  14:00: $124.02
  15:00: $125.01
  16:00: $124.92

2025-12-17:
 - Error: Error fetching intraday stock price for AAPL at 2025-12-17 10:00: No data found within 1 hour of 2025-12-17 10:00
 - Error: Error fetc

$AAPL: possibly delisted; no price data found  (1h 2025-12-17 -> 2025-12-20) (Yahoo error = "Data doesn't exist for startDate = 1765947600, endDate = 1766206800")


 - Error: Error fetching intraday stock price for AAPL at 2025-12-19 10:00: No intraday data available for AAPL around 2025-12-19 10:00


$AAPL: possibly delisted; no price data found  (1h 2025-12-17 -> 2025-12-20) (Yahoo error = "Data doesn't exist for startDate = 1765947600, endDate = 1766206800")


 - Error: Error fetching intraday stock price for AAPL at 2025-12-19 11:00: No intraday data available for AAPL around 2025-12-19 11:00


...

$AAPL: possibly delisted; no price data found  (1h 2025-12-30 -> 2026-01-02) (Yahoo error = "Data doesn't exist for startDate = 1767070800, endDate = 1767330000")


 - Error: Error fetching intraday stock price for AAPL at 2026-01-01 16:00: No intraday data available for AAPL around 2026-01-01 16:00

2026-01-02:


$AAPL: possibly delisted; no price data found  (1h 2025-12-31 -> 2026-01-03) (Yahoo error = "Data doesn't exist for startDate = 1767157200, endDate = 1767416400")


 - Error: Error fetching intraday stock price for AAPL at 2026-01-02 10:00: No intraday data available for AAPL around 2026-01-02 10:00


$AAPL: possibly delisted; no price data found  (1h 2025-12-31 -> 2026-01-03) (Yahoo error = "Data doesn't exist for startDate = 1767157200, endDate = 1767416400")
